Для обработки опубликованных и изданных текстов А. П. Чехова. При обработке неизданных/неопубликованных документов в исходном файле (TEIdoc) изменить информацию внутри тегов "isFinished" (оконченные/неокончнные) и "isPublished" (опубликованные/неопубликованные).

Исходные HTML-файлы должны быть размещены на гугл-диске (если они расположены в другом месте, поменять директорию. Переменные directory, new_directory).

При обработке томов, содержащих только один жанр текста (например, том 11 - пьесы) заполнить в исходном файле (TEIdoc) тег "textClass" (тип текста).

In [ ]:
from bs4 import BeautifulSoup
import re
!pip install natasha
import os

# Для документов, расположенных на гугл-диске
from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 34.4MB 112kB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 61kB 5.8MB/s 
     |████████████████████████████████| 51kB 5.2MB/s 
     |████████████████████████████████| 8.2MB 35.6MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26102 sha256=dcdbffa771721ae8b6d6ea51b26cc7fe10268368984f99193d1918a564ecb76f
  Stored in directory: /root/.cache/pip/wheels/f3/f2/66/e9c30d3e9499e65ea2fa0d07c002e64de63bd0adaa49c445bf
Successfully built intervaltree
  Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0
Mounted at /content/drive


In [ ]:
your_name = input('Введите свое ФИО в творительном падеже') # Заполняем имя того, кто преобразовывал файлы HTML в TEI

Введите свое ФИО в творительном падежеКудин Анастасией


In [ ]:
iter = 1 # Для нумерации текстов в названии файлов TEI

directory = 'drive/My Drive/Texts/3/' # Папка с файлами HTML
new_directory = 'drive/My Drive/Texts_tei/3/' # Папка для документов TEI

for filename in os.listdir(directory):

    file_html = open(directory + filename,'r',encoding='windows-1251')
    content = file_html.read()
    file_html.close()
    soup_html = BeautifulSoup(content, 'html.parser')

    file_tei = open('TEIdoc.xml','r',encoding='utf8') # Указать адрес исходного документа TEI
    content = file_tei.read()
    file_tei.close()
    soup_tei = BeautifulSoup(content, 'html.parser')

    notes_html = open('Notes3.html','r',encoding='windows-1251') # Указать адрес документа с примечаниями к обрабатываемому тому
    content1 = notes_html.read()
    notes_html.close()
    soup_notes_html = BeautifulSoup(content1, 'html.parser')

    # Заполняем заголовок
    title_main_tei = soup_tei.find('title')
    title_tag_html = soup_html.find_all('meta', attrs={'name':'title'})
    if len(title_tag_html) == 2: # Если есть подзаголовок
        title_main_tei.string = title_tag_html[0].get('content') + ' (' + title_tag_html[1].get('content') + ')'
    else:
        title_main_tei.string = title_tag_html[0].get('content')

    # Заполняем описание текста
    description_html = soup_html.find('div', class_='description')
    full_bibl = soup_tei.find('biblfull').find('p')
    description_html = soup_html.find('div', class_='description')
    full_bibl.string = description_html.text

    # Заполняем имя того, кто преобразовал текст в TEI
    resp_stmt = soup_tei.find('respstmt')
    name_resp_stmt = resp_stmt.find('persname')
    name_resp_stmt.string = your_name

    # Заполняем информацию об объеме произведения
    extent_inf = soup_tei.find('extent')

    list_ids = [] # Получаем список номеров страниц
    for tag in soup_html.find_all('span', class_='page') :
        tag_page=tag.get('id')
        list_ids.append(tag_page[2:])

    # Считаем объем
    volume = int(list_ids[-1]) - int(list_ids[0]) + 1
    volume_str = str(volume)

    # Заполняем информацию об объеме в тексте
    if volume_str[-1] == '5' or volume_str[-1] == '6' or volume_str[-1] == '7' or volume_str[-1] == '8' or volume_str[-1] == '9' or volume_str[-1] == '0' or volume_str[-2:] == '11' or volume_str[-2:] == '12' or volume_str[-2:] == '13' or volume_str[-2:] == '14' or volume_str[-2:] == '15' or volume_str[-2:] == '16' or volume_str[-2:] == '17' or volume_str[-2:] == '18' or volume_str[-2:] == '19':
        volume_f = volume_str + ' страниц'
    elif volume_str[-1] == '1':
        volume_f = volume_str + ' страница'
    elif volume_str[-1] == '2' or volume_str[-1] == '3' or volume_str[-1] == '4':
        volume_f = volume_str + ' страницы'

    # Заполняем информацию об объеме в атрибутах тегов
    tag_measure = soup_tei.new_tag('measure') 
    extent_inf.insert(0, tag_measure)

    soup_tei.find('measure')['unit'] = 'pages'
    soup_tei.find('measure')['quantity'] = volume_str

    measure_inf = soup_tei.find('measure')
    measure_inf.string = volume_f

    # Заполняем дату публикации
    publ_date = soup_tei.find('publicationstmt')
    publ_date1 = publ_date.find('date')
    date_1 = soup_html.find('meta', attrs={'name':'date'})
    publ_date1.string = date_1.get('content')
    publ_date.find('date')['when'] = date_1.get('content')

    # И дату создания
    date_cr = soup_tei.find('creation').find('date')
    creation = soup_tei.find('creation')
    abz = description_html.find_all('p')
    search = abz[-2].text

    date_from_to = re.findall(r'\d{4}—\d{4}', search)
    if len(date_from_to)>0: # Если том создавался несколько лет
        date_cr1 = date_from_to
        date_cr.string = date_cr1[0]
        creation.find('date')['from'] = date_cr1[0][:4]
        creation.find('date')['to'] = date_cr1[0][-4:]
    else: # Если том создавался один год (5 и 6 тома)
        date_when = re.findall(r'\d{4}', search)[0]
        creation.find('date')['when'] = date_when
        date_cr.string = date_when

    # Информация об издании
    edstmt = abz[0].text.split('//')
    edstmt1 = edstmt[1].split('\n')[0]
    edition_stmt = soup_tei.find('editionstmt').find('p')
    edition_stmt.string = edstmt1

    # Номер тома
    volume = re.findall(r'Т. \d+.', edition_stmt.string)
    volume_n = re.findall(r'\d+', volume[0])
    bibl_sc = soup_tei.find('biblscope')
    bibl_sc.string = 'Том ' + volume_n[0]

    # Получаем id нашего текста
    get_ids = []
    for link in soup_html.find_all('a'):
        get_ids.append(link.get('href'))
    text_id = get_ids[1].split('#')[1]

    # Находим примечания к тексту на странице с примечаниями
    first_tag = soup_notes_html.find('h4', attrs={'id':text_id})
    next_tag = first_tag.find_next('h4')
    all_tags = first_tag.find_all_next()

    # Выбираем текст, в котором будем искать именованные сущности (текст произведения + сноски)
    p = soup_html.find_all('p')
    text1 = ''
    for i in p:
        if i.has_attr('class'):
            text1 += i.text.split('\n')[0]
    for i in all_tags:
        if i != next_tag: 
            if i.has_attr('class') and re.match(r'small\S*', i['class'][0]) is not None:
                text1 += i.text.split('\n')[0]
        else:
            break

    # Ищем имена
    from natasha import (
        Segmenter,
        MorphVocab,
    
        NewsEmbedding,
        NewsMorphTagger,
        NewsSyntaxParser,
        NewsNERTagger,
    
        PER,
        NamesExtractor,
        DatesExtractor,
        MoneyExtractor,
        AddrExtractor,

        Doc
    )

    segmenter = Segmenter()
    morph_vocab = MorphVocab()

    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    syntax_parser = NewsSyntaxParser(emb)
    ner_tagger = NewsNERTagger(emb)

    names_extractor = NamesExtractor(morph_vocab)
    dates_extractor = DatesExtractor(morph_vocab)
    money_extractor = MoneyExtractor(morph_vocab)
    addr_extractor = AddrExtractor(morph_vocab)

    doc = Doc(text1)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)
    doc.tag_ner(ner_tagger)
    #names = []
    for span in doc.spans:
        span.normalize(morph_vocab)
    for span in doc.spans:
        if span.type == PER:
            span.extract_fact(names_extractor)
            names = [[_.text, _.fact.as_dict] for _ in doc.spans if _.fact]

    def delete_repeated(lst_names):
        res_names = []
        res_meta = []
        for name in lst_names:
            if name[0] not in res_names:
                res_names.append(name[0])
                res_meta.append(name[1])
        return list(zip(list(res_names), res_meta))

    def delete_postfixes(lst_names):
        names = set(map(lambda x: x[0], lst_names))
        for i in range(0, len(lst_names)-1):
            for j in range(i+1, len(lst_names)):
                if lst_names[i][0].startswith(lst_names[j][0]):
                    names.remove(lst_names[i][0])
                elif lst_names[j][0].startswith(lst_names[i][0]):
                    names.remove(lst_names[j][0])
        res = []
        for name in lst_names:
            if name[0] in names:
                res.append(name)
        return res

    names = delete_repeated(names)
    names = delete_postfixes(names)

    # Ищем текст произведения
    pars = soup_html.find_all(['p','span', 'img'])
    body = soup_tei.select('text body')[0]
    first_page = soup_html.find('span', class_='page').get('id')[2:]
    pb_tag = soup_tei.new_tag('pb', attrs={'n':first_page})

    # Получаем текст сносок
    text = soup_html.find_all('a')[1:]
    snos_list=[]
    for i in range(len(text)):
        a = text[i]
        if a.has_attr('class') and a['class'][0] == 'footnote':
            snos_list.append(a.text)
        elif a.has_attr('href') and re.match(r'#$\S*', a['href'][0]) is None:
            snos_list.append(a.text)
    snos_list = list(set(snos_list))

    # Переносим текст, примечания и имена
    for i in range(1,len(pars)):
        cur_tag = pars[i]

        if cur_tag.has_attr('class') and cur_tag['class'][0] == 'page': # Если это страница
            cur_page = cur_tag.get('id')[2:]
            pb_tag = soup_tei.new_tag('pb', attrs={'n':cur_page})
            pb_tag.string = cur_tag.get('id')[2:]
            body.append(pb_tag)

        elif cur_tag.has_attr('alt'): # Если это изображение
            fig_tag = soup_tei.new_tag('figure')
            pb_tag.append(fig_tag)
            graph_tag = soup_tei.new_tag('graphic')
            fig_tag.append(graph_tag)
            get_href = cur_tag.get('src').split('/')
            fig_tag.find('graphic')['url'] = 'http://feb-web.ru/feb/chekhov/' + get_href[-2] + '/' + get_href[-1]
            desc_tag = soup_tei.new_tag('figDesc')
            fig_tag.append(desc_tag)
            desc_tag.string = cur_tag.get('alt')
            if cur_tag.find('a') is not None: # Если подпись изображения содержит примечание
                for i in range(len(snos_list)):
                    note_next_el = desc_tag.string[desc_tag.string.find(snos_list[i][-1])+1]
                    note_prev_el = desc_tag.string[desc_tag.string.find(snos_list[i][0])-1]
                    if note_prev_el != ' ':
                        if note_next_el == ' ' or note_next_el == '.' or note_next_el == ',' or note_next_el == '!' or note_next_el == ':' or note_next_el == ';' or note_next_el == '?' or note_next_el == ')':
                            desc_tag.string = desc_tag.string.replace(snos_list[i], '<note>'+snos_list[i]+'</note>')
                    elif snos_list[i][0] != '1' and snos_list[i][0] != '2' and snos_list[i][0] != '3' and snos_list[i][0] != '4' and snos_list[i][0] != '5' and snos_list[i][0] != '6' and snos_list[i][0] != '7' and snos_list[i][0] != '8' and snos_list[i][0] != '9':
                        desc_tag.string = desc_tag.string.replace(snos_list[i], '<note>'+snos_list[i]+'</note>')

            for i in range(len(names)): # Если в подписи изображения есть имена
                if names[i][0] in desc_tag.string:
                    desc_tag.string = desc_tag.string.replace(names[i], '<PersName type="figure">'+names[i]+'</PersName>')
                    #desc_tag.string = re.sub(names[i][0][:-1] + r'(\w*)', '<PersName type="figure">'+names[i][0][:-1] + r'\g<1>' +'</PersName>', desc_tag.string)
                    if names[i][1].get('first') is not None:
                        if len(names[i][1].get('first')) == 1:
                            desc_tag.string = re.sub(names[i][1].get('first') + '.', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first') + '.' +'</forename>', desc_tag.string)
                        else:
                            desc_tag.string = re.sub(names[i][1].get('first')[:-1] + r'(\w*)', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first')[:-1] + r'\g<1>' +'</forename>', desc_tag.string)
                    if names[i][1].get('last') is not None:
                        if len(names[i][1].get('last')) == 1:
                            desc_tag.string = re.sub(names[i][1].get('last') + '.', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last') + '.' +'</surname>', desc_tag.string)
                        else:
                            desc_tag.string = re.sub(names[i][1].get('last')[:-1] + r'(\w*)', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last')[:-1] + r'\g<1>' +'</surname>', desc_tag.string)
                    if names[i][1].get('middle') is not None:
                        if len(names[i][1].get('middle')) == 1:
                            desc_tag.string = re.sub(names[i][1].get('middle') + '.', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle') + '.' +'</forename>', desc_tag.string)
                        else:
                            desc_tag.string = re.sub(names[i][1].get('middle')[:-1] + r'(\w*)', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle')[:-1] + r'\g<1>' +'</forename>', desc_tag.string)
                
            dates = list(dates_extractor(desc_tag.string)) # Если в подписи изображения есть даты
            if len(dates) > 0:
                for i in range(len(dates)-1, -1, -1):
                    result = re.findall(r'\d+', str(dates[i]))
                    if len(result) == 3:
                        desc_tag.string = desc_tag.string.replace(desc_tag.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '">' + desc_tag.string[int(result[0]):int(result[1])] + '</date>')
                    elif len(result) == 4:
                        desc_tag.string = desc_tag.string.replace(desc_tag.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '">' + desc_tag.string[int(result[0]):int(result[1])] + '</date>')
                    elif len(result) == 5:
                        desc_tag.string = desc_tag.string.replace(desc_tag.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '-' + result[4] + '">' + desc_tag.string[int(result[0]):int(result[1])] + '</date>')

        elif cur_tag.has_attr('class') and re.match(r'zg\S*', cur_tag['class'][0]) is not None: # Если это подзаголовок
            new_p = soup_tei.new_tag('p')
            pb_tag.append(new_p)
            tag_headline = soup_tei.new_tag('head')
            new_p.append(tag_headline)
            body.find('head')['rend'] = 'center'
            body.find('head')['type'] = 'subtitle'
            tag_headline.string = cur_tag.text.split('\n')[0]
            if cur_tag.find('a') is not None: # Если подзаголовок содержит примечание
                for i in range(len(snos_list)):
                    if snos_list[i] in tag_headline.string:
                        tag_headline.string = tag_headline.string.replace(snos_list[i], '<note>'+snos_list[i]+'</note>')

            for i in range(len(names)): # Если в подзаголовке есть имена
                if names[i][0] in tag_headline.string:
                    tag_headline.string = tag_headline.string.replace(i, '<PersName type="subtitle">'+i+'</PersName>')
                    #tag_headline.string = re.sub(names[i][0][:-1] + r'(\w*)', '<PersName type="subtitle">'+names[i][0][:-1] + r'\g<1>' +'</PersName>', tag_headline.string)
                    if names[i][1].get('first') is not None:
                        if len(names[i][1].get('first')) == 1:
                            tag_headline.string = re.sub(names[i][1].get('first') + '.', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first') + '.' +'</forename>', tag_headline.string)
                        else:
                            tag_headline.string = re.sub(names[i][1].get('first')[:-1] + r'(\w*)', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first')[:-1] + r'\g<1>' +'</forename>', tag_headline.string)
                    if names[i][1].get('last') is not None:
                        if len(names[i][1].get('last')) == 1:
                            tag_headline.string = re.sub(names[i][1].get('last') + '.', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last') + '.' +'</surname>', tag_headline.string)
                        else:
                            tag_headline.string = re.sub(names[i][1].get('last')[:-1] + r'(\w*)', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last')[:-1] + r'\g<1>' +'</surname>', tag_headline.string)
                    if names[i][1].get('middle') is not None:
                        if len(names[i][1].get('middle')) == 1:
                            tag_headline.string = re.sub(names[i][1].get('middle') + '.', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle') + '.' +'</forename>', tag_headline.string)
                        else:
                            tag_headline.string = re.sub(names[i][1].get('middle')[:-1] + r'(\w*)', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle')[:-1] + r'\g<1>' +'</forename>', tag_headline.string)

            dates = list(dates_extractor(tag_headline.string)) # Если в подзаголовке есть даты
            if len(dates) > 0:
                for i in range(len(dates)-1, -1, -1):
                    result = re.findall(r'\d+', str(dates[i]))
                    if len(result) == 3:
                        tag_headline.string = tag_headline.string.replace(tag_headline.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '">' + tag_headline.string[int(result[0]):int(result[1])] + '</date>')
                    elif len(result) == 4:
                        tag_headline.string = tag_headline.string.replace(tag_headline.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '">' + tag_headline.string[int(result[0]):int(result[1])] + '</date>')
                    elif len(result) == 5:
                        tag_headline.string = tag_headline.string.replace(tag_headline.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '-' + result[4] + '">' + tag_headline.string[int(result[0]):int(result[1])] + '</date>')

        elif cur_tag.has_attr('class') and re.match(r'tit\S*|zag\S*', cur_tag['class'][0]) is not None: # Если это заголовок
            new_p = soup_tei.new_tag('p')
            pb_tag.append(new_p)
            tag_headline = soup_tei.new_tag('head')
            new_p.append(tag_headline)
            body.find('head')['rend'] = 'center'
            tag_hi = soup_tei.new_tag('hi') 
            tag_headline.append(tag_hi)
            body.find('hi')['rend'] = 'strong'
            tag_hi.string = cur_tag.text.split('\n')[0]
            if cur_tag.find('a') is not None: # Если заголовок содержит примечание
                for i in range(len(snos_list)):
                    if snos_list[i] in tag_hi.string:
                        tag_hi.string = tag_hi.string.replace(snos_list[i], '<note>'+snos_list[i]+'</note>')
        
            for i in range(len(names)): # Если в заголовке есть имена
                if names[i][0] in tag_hi.string:
                    tag_hi.string = tag_hi.string.replace(i, '<PersName type="title">'+i+'</PersName>')
                    #tag_hi.string = re.sub(names[i][0][:-1] + r'(\w*)', '<PersName type="title">'+names[i][0][:-1] + r'\g<1>' +'</PersName>', tag_hi.string)
                    if names[i][1].get('first') is not None:
                        if len(names[i][1].get('first')) == 1:
                            tag_hi.string = re.sub(names[i][1].get('first') + '.', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first') + '.' +'</forename>', tag_hi.string)
                        else:
                            tag_hi.string = re.sub(names[i][1].get('first')[:-1] + r'(\w*)', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first')[:-1] + r'\g<1>' +'</forename>', tag_hi.string)
                    if names[i][1].get('last') is not None:
                        if len(names[i][1].get('last')) == 1:
                            tag_hi.string = re.sub(names[i][1].get('last') + '.', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last') + '.' +'</surname>', tag_hi.string)
                        else:
                            tag_hi.string = re.sub(names[i][1].get('last')[:-1] + r'(\w*)', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last')[:-1] + r'\g<1>' +'</surname>', tag_hi.string)
                    if names[i][1].get('middle') is not None:
                        if len(names[i][1].get('middle')) == 1:
                            tag_hi.string = re.sub(names[i][1].get('middle') + '.', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle') + '.' +'</forename>', tag_hi.string)
                        else:
                            tag_hi.string = re.sub(names[i][1].get('middle')[:-1] + r'(\w*)', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle')[:-1] + r'\g<1>' +'</forename>', tag_hi.string)
        
            dates = list(dates_extractor(tag_hi.string)) # Если в заголовке есть даты
            if len(dates) > 0:
                for i in range(len(dates)-1, -1, -1):
                    result = re.findall(r'\d+', str(dates[i]))
                    if len(result) == 3:
                        tag_hi.string = tag_hi.string.replace(tag_hi.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '">' + tag_hi.string[int(result[0]):int(result[1])] + '</date>')
                    elif len(result) == 4:
                        tag_hi.string = tag_hi.string.replace(tag_hi.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '">' + tag_hi.string[int(result[0]):int(result[1])] + '</date>')
                    elif len(result) == 5:
                        tag_hi.string = tag_hi.string.replace(tag_hi.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '-' + result[4] + '">' + tag_hi.string[int(result[0]):int(result[1])] + '</date>')

        elif cur_tag.has_attr('class') and re.match(r'epig\S*', cur_tag['class'][0]) is not None: # Если это эпиграф
            new_p = soup_tei.new_tag('p')
            pb_tag.append(new_p)
            tag_epigraph = soup_tei.new_tag('epigraph')
            new_p.append(tag_epigraph)
            body.find('epigraph')['rend'] = 'right'
            tag_epigraph.string = cur_tag.text
            if cur_tag.find('a') is not None: # Если эпиграф содержит примечание
                for i in range(len(snos_list)):
                    note_next_el = tag_epigraph.string[tag_epigraph.string.find(snos_list[i][-1])+1]
                    note_prev_el = tag_epigraph.string[tag_epigraph.string.find(snos_list[i][0])-1]
                    if note_prev_el != ' ':
                        if note_next_el == ' ' or note_next_el == '.' or note_next_el == ',' or note_next_el == '!' or note_next_el == ':' or note_next_el == ';' or note_next_el == '?' or note_next_el == ')':
                            tag_epigraph.string = tag_epigraph.string.replace(snos_list[i], '<note>'+snos_list[i]+'</note>')
                    elif snos_list[i][0] != '1' and snos_list[i][0] != '2' and snos_list[i][0] != '3' and snos_list[i][0] != '4' and snos_list[i][0] != '5' and snos_list[i][0] != '6' and snos_list[i][0] != '7' and snos_list[i][0] != '8' and snos_list[i][0] != '9':
                        tag_epigraph.string = tag_epigraph.string.replace(snos_list[i], '<note>'+snos_list[i]+'</note>')
        
            for i in range(len(names)): # Если в эпиграфе есть имена
                if names[i][0] in tag_epigraph.string:
                    tag_epigraph.string = tag_epigraph.string.replace(names[i][0], '<PersName type="epigraph">'+names[i][0]+'</PersName>')
                    #tag_epigraph.string = re.sub(names[i][0][:-1] + r'(\w*)', '<PersName type="epigraph">'+names[i][0][:-1] + r'\g<1>' +'</PersName>', tag_epigraph.string)
                    if names[i][1].get('first') is not None:
                        if len(names[i][1].get('first')) == 1:
                            tag_epigraph.string = re.sub(names[i][1].get('first') + '.', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first') + '.' +'</forename>', tag_epigraph.string)
                        else:
                            tag_epigraph.string = re.sub(names[i][1].get('first')[:-1] + r'(\w*)', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first')[:-1] + r'\g<1>' +'</forename>', tag_epigraph.string)
                    if names[i][1].get('last') is not None:
                        if len(names[i][1].get('last')) == 1:
                            tag_epigraph.string = re.sub(names[i][1].get('last') + '.', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last') + '.' +'</surname>', tag_epigraph.string)
                        else:
                            tag_epigraph.string = re.sub(names[i][1].get('last')[:-1] + r'(\w*)', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last')[:-1] + r'\g<1>' +'</surname>', tag_epigraph.string)
                    if names[i][1].get('middle') is not None:
                        if len(names[i][1].get('middle')) == 1:
                            tag_epigraph.string = re.sub(names[i][1].get('middle') + '.', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle') + '.' +'</forename>', tag_epigraph.string)
                        else:
                            tag_epigraph.string = re.sub(names[i][1].get('middle')[:-1] + r'(\w*)', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle')[:-1] + r'\g<1>' +'</forename>', tag_epigraph.string)
        
            dates = list(dates_extractor(tag_epigraph.string)) # Если в эпиграфе есть даты
            if len(dates) > 0:
                for i in range(len(dates)-1, -1, -1):
                    result = re.findall(r'\d+', str(dates[i]))
                    if len(result) == 3:
                        tag_epigraph.string = tag_epigraph.string.replace(tag_epigraph.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '">' + tag_epigraph.string[int(result[0]):int(result[1])] + '</date>')
                    elif len(result) == 4:
                        tag_epigraph.string = tag_epigraph.string.replace(tag_epigraph.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '">' + tag_epigraph.string[int(result[0]):int(result[1])] + '</date>')
                    elif len(result) == 5:
                        tag_epigraph.string = tag_epigraph.string.replace(tag_epigraph.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '-' + result[4] + '">' + tag_epigraph.string[int(result[0]):int(result[1])] + '</date>')

        elif cur_tag.has_attr('class') and re.match(r'obraw\S*', cur_tag['class'][0]) is not None: # Если это приветствие/обращение (в письме)
            new_p = soup_tei.new_tag('p')
            pb_tag.append(new_p)
            tag_salute = soup_tei.new_tag('salute')
            new_p.append(tag_salute)
            tag_salute.string = cur_tag.text.split('\n')[0]
            if cur_tag.find('a') is not None: # Если обращение содержит примечание
                for i in range(len(snos_list)):
                    note_next_el = tag_salute.string[tag_salute.string.find(snos_list[i][-1])+1]
                    note_prev_el = tag_salute.string[tag_salute.string.find(snos_list[i][0])-1]
                    if note_prev_el != ' ':
                        if note_next_el == ' ' or note_next_el == '.' or note_next_el == ',' or note_next_el == '!' or note_next_el == ':' or note_next_el == ';' or note_next_el == '?' or note_next_el == ')':
                            tag_salute.string = tag_salute.string.replace(snos_list[i], '<note>'+snos_list[i]+'</note>')
                    elif snos_list[i][0] != '1' and snos_list[i][0] != '2' and snos_list[i][0] != '3' and snos_list[i][0] != '4' and snos_list[i][0] != '5' and snos_list[i][0] != '6' and snos_list[i][0] != '7' and snos_list[i][0] != '8' and snos_list[i][0] != '9':
                        tag_salute.string = tag_salute.string.replace(snos_list[i], '<note>'+snos_list[i]+'</note>')
        
            for i in range(len(names)): # Если в обращении есть имена
                if names[i][0] in tag_salute.string:
                    tag_salute.string = tag_salute.string.replace(names[i][0], '<PersName type="salute">'+names[i][0]+'</PersName>')
                    #tag_salute.string = re.sub(names[i][0][:-1] + r'(\w*)', '<PersName type="salute">'+names[i][0][:-1] + r'\g<1>' +'</PersName>', tag_salute.string)
                    if names[i][1].get('first') is not None:
                        if len(names[i][1].get('first')) == 1:
                            tag_salute.string = re.sub(names[i][1].get('first') + '.', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first') + '.' +'</forename>', tag_salute.string)
                        else:
                            tag_salute.string = re.sub(names[i][1].get('first')[:-1] + r'(\w*)', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first')[:-1] + r'\g<1>' +'</forename>', tag_salute.string)
                    if names[i][1].get('last') is not None:
                        if len(names[i][1].get('last')) == 1:
                            tag_salute.string = re.sub(names[i][1].get('last') + '.', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last') + '.' +'</surname>', tag_salute.string)
                        else:
                            tag_salute.string = re.sub(names[i][1].get('last')[:-1] + r'(\w*)', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last')[:-1] + r'\g<1>' +'</surname>', tag_salute.string)
                    if names[i][1].get('middle') is not None:
                        if len(names[i][1].get('middle')) == 1:
                            tag_salute.string = re.sub(names[i][1].get('middle') + '.', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle') + '.' +'</forename>', tag_salute.string)
                        else:
                            tag_salute.string = re.sub(names[i][1].get('middle')[:-1] + r'(\w*)', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle')[:-1] + r'\g<1>' +'</forename>', tag_salute.string)
        
            dates = list(dates_extractor(tag_salute.string)) # Если в обращении есть даты
            if len(dates) > 0:
                for i in range(len(dates)-1, -1, -1):
                    result = re.findall(r'\d+', str(dates[i]))
                    if len(result) == 3:
                        tag_salute.string = tag_salute.string.replace(tag_salute.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '">' + tag_salute.string[int(result[0]):int(result[1])] + '</date>')
                    elif len(result) == 4:
                        tag_salute.string = tag_salute.string.replace(tag_salute.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '">' + tag_salute.string[int(result[0]):int(result[1])] + '</date>')
                    elif len(result) == 5:
                        tag_salute.string = tag_salute.string.replace(tag_salute.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '-' + result[4] + '">' + tag_salute.string[int(result[0]):int(result[1])] + '</date>')

        elif cur_tag.has_attr('class') and re.match(r'podp\S*', cur_tag['class'][0]) is not None: # Если это подпись (в письме)
            new_p = soup_tei.new_tag('p')
            pb_tag.append(new_p)
            tag_podp = soup_tei.new_tag('signed')
            new_p.append(tag_podp)
            tag_podp.string = cur_tag.text.split('\n')[0]
            if cur_tag.find('a') is not None: # Если подпись содержит примечание
                for i in range(len(snos_list)):
                    note_next_el = tag_podp.string[tag_podp.string.find(snos_list[i][-1])+1]
                    note_prev_el = tag_podp.string[tag_podp.string.find(snos_list[i][0])-1]
                    if note_prev_el != ' ':
                        if note_next_el == ' ' or note_next_el == '.' or note_next_el == ',' or note_next_el == '!' or note_next_el == ':' or note_next_el == ';' or note_next_el == '?' or note_next_el == ')':
                            tag_podp.string = tag_podp.string.replace(snos_list[i], '<note>'+snos_list[i]+'</note>')
                    elif snos_list[i][0] != '1' and snos_list[i][0] != '2' and snos_list[i][0] != '3' and snos_list[i][0] != '4' and snos_list[i][0] != '5' and snos_list[i][0] != '6' and snos_list[i][0] != '7' and snos_list[i][0] != '8' and snos_list[i][0] != '9':
                        tag_podp.string = tag_podp.string.replace(snos_list[i], '<note>'+snos_list[i]+'</note>')
        
            for i in range(len(names)): # Если в подписи есть имена
                if names[i][0] in tag_podp.string:
                    tag_podp.string = tag_podp.string.replace(names[i][0], '<PersName type="signed">'+names[i][0]+'</PersName>')
                    #tag_podp.string = re.sub(names[i][0][:-1] + r'(\w*)', '<PersName type="signed">'+names[i][0][:-1] + r'\g<1>' +'</PersName>', tag_podp.string)
                    if names[i][1].get('first') is not None:
                        if len(names[i][1].get('first')) == 1:
                            tag_podp.string = re.sub(names[i][1].get('first') + '.', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first') + '.' +'</forename>', tag_podp.string)
                        else:
                            tag_podp.string = re.sub(names[i][1].get('first')[:-1] + r'(\w*)', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first')[:-1] + r'\g<1>' +'</forename>', tag_podp.string)
                    if names[i][1].get('last') is not None:
                        if len(names[i][1].get('last')) == 1:
                            tag_podp.string = re.sub(names[i][1].get('last') + '.', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last') + '.' +'</surname>', tag_podp.string)
                        else:
                            tag_podp.string = re.sub(names[i][1].get('last')[:-1] + r'(\w*)', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last')[:-1] + r'\g<1>' +'</surname>', tag_podp.string)
                    if names[i][1].get('middle') is not None:
                        if len(names[i][1].get('middle')) == 1:
                            tag_podp.string = re.sub(names[i][1].get('middle') + '.', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle') + '.' +'</forename>', tag_podp.string)
                        else:
                            tag_podp.string = re.sub(names[i][1].get('middle')[:-1] + r'(\w*)', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle')[:-1] + r'\g<1>' +'</forename>', tag_podp.string)
        
            dates = list(dates_extractor(tag_podp.string)) # Если в подписи есть даты
            if len(dates) > 0:
                for i in range(len(dates)-1, -1, -1):
                    result = re.findall(r'\d+', str(dates[i]))
                    if len(result) == 3:
                        tag_podp.string = tag_podp.string.replace(tag_podp.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '">' + tag_podp.string[int(result[0]):int(result[1])] + '</date>')
                    elif len(result) == 4:
                        tag_podp.string = tag_podp.string.replace(tag_podp.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '">' + tag_podp.string[int(result[0]):int(result[1])] + '</date>')
                    elif len(result) == 5:
                        tag_podp.string = tag_podp.string.replace(tag_podp.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '-' + result[4] + '">' + tag_podp.string[int(result[0]):int(result[1])] + '</date>')

        elif cur_tag.has_attr('class') and re.match(r'text\S*|curs\S*|small\S*|vis\S*|center\S*', cur_tag['class'][0]) is not None: # Если это текст
            new_p = soup_tei.new_tag('p')
            pb_tag.append(new_p) 
            new_p.string = cur_tag.text.split('\n')[0]
            if cur_tag.find('a') is not None: # Размечаем сноски внутри текста
                for i in range(len(snos_list)):
                    note_next_el = new_p.string[new_p.string.find(snos_list[i][-1])+1]
                    note_prev_el = new_p.string[new_p.string.find(snos_list[i][0])-1]
                    if note_prev_el != ' ':
                        if note_next_el == ' ' or note_next_el == '.' or note_next_el == ',' or note_next_el == '!' or note_next_el == ':' or note_next_el == ';' or note_next_el == '?' or note_next_el == ')':
                            new_p.string = new_p.string.replace(snos_list[i], '<note>'+snos_list[i]+'</note>')
                    elif snos_list[i][0] != '1' and snos_list[i][0] != '2' and snos_list[i][0] != '3' and snos_list[i][0] != '4' and snos_list[i][0] != '5' and snos_list[i][0] != '6' and snos_list[i][0] != '7' and snos_list[i][0] != '8' and snos_list[i][0] != '9':
                        new_p.string = new_p.string.replace(snos_list[i], '<note>'+snos_list[i]+'</note>')
        
            for i in range(len(names)): # Если в тексте есть имена
                if names[i][0] in new_p.string:
                    new_p.string = new_p.string.replace(names[i][0], '<PersName type="text">'+names[i][0]+'</PersName>')
                    #new_p.string = re.sub(names[i][0][:-1] + r'(\w*)', '<PersName type="text">'+names[i][0][:-1] + r'\g<1>' +'</PersName>', new_p.string)
                    if names[i][1].get('first') is not None:
                        if len(names[i][1].get('first')) == 1:
                            new_p.string = re.sub(names[i][1].get('first') + '.', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first') + '.' +'</forename>', new_p.string)
                        else:
                            new_p.string = re.sub(names[i][1].get('first')[:-1] + r'(\w*)', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first')[:-1] + r'\g<1>' +'</forename>', new_p.string)
                    if names[i][1].get('last') is not None:
                        if len(names[i][1].get('last')) == 1:
                            new_p.string = re.sub(names[i][1].get('last') + '.', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last') + '.' +'</surname>', new_p.string)
                        else:
                            new_p.string = re.sub(names[i][1].get('last')[:-1] + r'(\w*)', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last')[:-1] + r'\g<1>' +'</surname>', new_p.string)
                    if names[i][1].get('middle') is not None:
                        if len(names[i][1].get('middle')) == 1:
                            new_p.string = re.sub(names[i][1].get('middle') + '.', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle') + '.' +'</forename>', new_p.string)
                        else:
                             new_p.string = re.sub(names[i][1].get('middle')[:-1] + r'(\w*)', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle')[:-1] + r'\g<1>' +'</forename>', new_p.string)
        
            dates = list(dates_extractor(new_p.string)) # Если в тексте есть даты
            if len(dates) > 0:
                for i in range(len(dates)-1, -1, -1):
                    result = re.findall(r'\d+', str(dates[i]))
                    if len(result) == 3:
                        new_p.string = new_p.string.replace(new_p.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '">' + new_p.string[int(result[0]):int(result[1])] + '</date>')
                    elif len(result) == 4:
                        new_p.string = new_p.string.replace(new_p.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '">' + new_p.string[int(result[0]):int(result[1])] + '</date>')
                    elif len(result) == 5:
                        new_p.string = new_p.string.replace(new_p.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '-' + result[4] + '">' + new_p.string[int(result[0]):int(result[1])] + '</date>')

    # Переносим сноски
    link_tag = soup_tei.new_tag('linkGrp')
    body.append(link_tag)
    link_tag.string = 'Примечания'

    note_tag = soup_tei.new_tag('link')
    link_tag.append(note_tag)

    for i in all_tags:
        if i != next_tag: 
            if i.has_attr('class') and re.match(r'small\S*|prim\S*|text\S*|txt\S*|mtext\S*', i['class'][0]) is not None:
                if i.text[0:4] != 'Стр.':
                    new_p = soup_tei.new_tag('p')
                    note_tag.append(new_p)
                    new_p.string = i.text.split('\n')[0]
                    for j in range(len(names)): # Если в примечании есть имена
                        if names[j][0] in new_p.string:
                            new_p.string = new_p.string.replace(names[j][0], '<PersName type="note">'+names[j][0]+'</PersName>')
                            #new_p.string = re.sub(names[j][0][:-1] + r'(\w*)', '<PersName type="note">'+names[j][0][:-1] + r'\g<1>' +'</PersName>', new_p.string)
                            if names[j][1].get('first') is not None:
                                if len(names[j][1].get('first')) == 1:
                                    new_p.string = re.sub(names[j][1].get('first') + '.', '<forename xml:id="' + names[j][1].get('first') + '">'+ names[j][1].get('first') + '.' +'</forename>', new_p.string)
                                else:
                                    new_p.string = re.sub(names[j][1].get('first')[:-1] + r'(\w*)', '<forename xml:id="' + names[j][1].get('first') + '">'+ names[j][1].get('first')[:-1] + r'\g<1>' +'</forename>', new_p.string)
                            if names[j][1].get('last') is not None:
                                if len(names[j][1].get('last')) == 1:
                                    new_p.string = re.sub(names[j][1].get('last') + '.', '<surname xml:id="' + names[j][1].get('last') + '">'+ names[j][1].get('last') + '.' +'</surname>', new_p.string)
                                else:
                                    new_p.string = re.sub(names[j][1].get('last')[:-1] + r'(\w*)', '<surname xml:id="' + names[j][1].get('last') + '">'+ names[j][1].get('last')[:-1] + r'\g<1>' +'</surname>', new_p.string)
                            if names[j][1].get('middle') is not None:
                                if len(names[j][1].get('middle')) == 1:
                                    new_p.string = re.sub(names[j][1].get('middle') + '.', '<forename type="patronym" xml:id="' + names[j][1].get('middle') + '">'+ names[j][1].get('middle') + '.' +'</forename>', new_p.string)
                                else:
                                    new_p.string = re.sub(names[j][1].get('middle')[:-1] + r'(\w*)', '<forename type="patronym" xml:id="' + names[j][1].get('middle') + '">'+ names[j][1].get('middle')[:-1] + r'\g<1>' +'</forename>', new_p.string)
                
                    dates = list(dates_extractor(new_p.string)) # Если в примечании есть даты
                    if len(dates) > 0:
                        for j in range(len(dates)-1, -1, -1):
                            result = re.findall(r'\d+', str(dates[j]))
                            if len(result) == 3:
                                new_p.string = new_p.string.replace(new_p.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '">' + new_p.string[int(result[0]):int(result[1])] + '</date>')
                            elif len(result) == 4:
                                new_p.string = new_p.string.replace(new_p.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '">' + new_p.string[int(result[0]):int(result[1])] + '</date>')
                            elif len(result) == 5:
                                new_p.string = new_p.string.replace(new_p.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '-' + result[4] + '">' + new_p.string[int(result[0]):int(result[1])] + '</date>')
            
                elif i.text[0:4] == 'Стр.':
                    note_tag1 = soup_tei.new_tag('link')
                    link_tag.append(note_tag1)
                    new_p = soup_tei.new_tag('p')
                    note_tag1.append(new_p)
                    new_p.string = i.text.split('\n')[0]
                
                    for i in range(len(names)): # Если в примечании есть имена #2
                        if names[i][0] in new_p.string:
                            new_p.string = new_p.string.replace(names[i][0], '<PersName type="note">'+names[i][0]+'</PersName>')
                            #new_p.string = re.sub(names[i][0][:-1] + r'(\w*)', '<PersName type="note">'+names[i][0][:-1] + r'\g<1>' +'</PersName>', new_p.string)
                            if names[i][1].get('first') is not None:
                                if len(names[i][1].get('first')) == 1:
                                    new_p.string = re.sub(names[i][1].get('first') + '.', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first') + '.' +'</forename>', new_p.string)
                                else:
                                    new_p.string = re.sub(names[i][1].get('first')[:-1] + r'(\w*)', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first')[:-1] + r'\g<1>' +'</forename>', new_p.string)
                            if names[i][1].get('last') is not None:
                                if len(names[i][1].get('last')) == 1:
                                    new_p.string = re.sub(names[i][1].get('last') + '.', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last') + '.' +'</surname>', new_p.string)
                                else:
                                    new_p.string = re.sub(names[i][1].get('last')[:-1] + r'(\w*)', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last')[:-1] + r'\g<1>' +'</surname>', new_p.string)
                            if names[i][1].get('middle') is not None:
                                if len(names[i][1].get('middle')) == 1:
                                    new_p.string = re.sub(names[i][1].get('middle') + '.', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle') + '.' +'</forename>', new_p.string)
                                else:
                                    new_p.string = re.sub(names[i][1].get('middle')[:-1] + r'(\w*)', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle')[:-1] + r'\g<1>' +'</forename>', new_p.string)
                
                    dates = list(dates_extractor(new_p.string)) # Если в примечании есть даты
                    if len(dates) > 0:
                        for i in range(len(dates)-1, -1, -1):
                            result = re.findall(r'\d+', str(dates[i]))
                            if len(result) == 3:
                                new_p.string = new_p.string.replace(new_p.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '">' + new_p.string[int(result[0]):int(result[1])] + '</date>')
                            elif len(result) == 4:
                                new_p.string = new_p.string.replace(new_p.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '">' + new_p.string[int(result[0]):int(result[1])] + '</date>')
                            elif len(result) == 5:
                                new_p.string = new_p.string.replace(new_p.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '-' + result[4] + '">' + new_p.string[int(result[0]):int(result[1])] + '</date>')
            
        else:
            break

    # Получаем примечания внутри произведения из раздела сноски
    text = soup_html.find_all('p', class_='snos')
    for i in range(len(text)):
        a = text[i]
        note_tag = soup_tei.new_tag('link')
        link_tag.append(note_tag)
        note_tag.string = a.text
  
        for i in range(len(names)): # Если в примечании есть имена #3
            if names[i][0] in note_tag.string:
                note_tag.string = note_tag.string.replace(names[i][0], '<PersName type="note">'+names[i][0]+'</PersName>')
                #note_tag.string = re.sub(names[i][0][:-1] + r'(\w*)', '<PersName type="note">'+names[i][0][:-1] + r'\g<1>' +'</PersName>', note_tag.string)
                if names[i][1].get('first') is not None:
                    if len(names[i][1].get('first')) == 1:
                        note_tag.string = re.sub(names[i][1].get('first') + '.', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first') + '.' +'</forename>', note_tag.string)
                    else:
                        note_tag.string = re.sub(names[i][1].get('first')[:-1] + r'(\w*)', '<forename xml:id="' + names[i][1].get('first') + '">'+ names[i][1].get('first')[:-1] + r'\g<1>' +'</forename>', note_tag.string)
                if names[i][1].get('last') is not None:
                    if len(names[i][1].get('last')) == 1:
                        note_tag.string = re.sub(names[i][1].get('last') + '.', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last') + '.' +'</surname>', note_tag.string)
                    else:
                        note_tag.string = re.sub(names[i][1].get('last')[:-1] + r'(\w*)', '<surname xml:id="' + names[i][1].get('last') + '">'+ names[i][1].get('last')[:-1] + r'\g<1>' +'</surname>', note_tag.string)
                if names[i][1].get('middle') is not None:
                    if len(names[i][1].get('middle')) == 1:
                        note_tag.string = re.sub(names[i][1].get('middle') + '.', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle') + '.' +'</forename>', note_tag.string)
                    else:
                        note_tag.string = re.sub(names[i][1].get('middle')[:-1] + r'(\w*)', '<forename type="patronym" xml:id="' + names[i][1].get('middle') + '">'+ names[i][1].get('middle')[:-1] + r'\g<1>' +'</forename>', note_tag.string)
  
        dates = list(dates_extractor(note_tag.string)) # Если в примечании есть даты
        if len(dates) > 0:
            for i in range(len(dates)-1, -1, -1):
                result = re.findall(r'\d+', str(dates[i]))
                if len(result) == 3:
                    note_tag.string = note_tag.string.replace(note_tag.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '">' + note_tag.string[int(result[0]):int(result[1])] + '</date>')
                elif len(result) == 4:
                    note_tag.string = note_tag.string.replace(note_tag.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '">' + note_tag.string[int(result[0]):int(result[1])] + '</date>')
                elif len(result) == 5:
                    note_tag.string = note_tag.string.replace(note_tag.string[int(result[0]):int(result[1])], '<date when="' + result[2] + '-' + result[3] + '-' + result[4] + '">' + note_tag.string[int(result[0]):int(result[1])] + '</date>')

    tei_doc = str(soup_tei.prettify()[2:])
    tei_doc = tei_doc.replace('&lt;', '<')
    tei_doc = tei_doc.replace('&gt;', '>')

    file = open('done.xml', 'w', encoding='utf8')
    file.write(tei_doc)
    file.close()

    # Присваиваем примечаниям id
    file_tei = open('done.xml','r',encoding='utf8')
    content = file_tei.read()
    file_tei.close()
    soup_tei = BeautifulSoup(content, 'html.parser')

    body = soup_tei.find('body')
    notes = body.find_all('note')
    count = 1
    for i in range(len(notes)):
        if re.match(r'\d+', notes[i].text) is not None:
            notes[i]['xml:id'] = 'note'+str(count)
            count += 1

    link_gr = soup_tei.find('linkgrp')
    links = link_gr.find_all('link')[-len(notes)+1:]
    cnt = 1
    for i in range(len(links)):
        if re.match(r'\d+', links[i].text) is not None:
            links[i]['target'] = '#note'+str(cnt)
            cnt += 1

    for i in range(len(notes)):
        if notes[i].has_attr('xml:id') is False:
            notes[i]['xml:id'] = 'note'+str(count)
            count += 1

    links = link_gr.find_all('link')

    for i in range(len(links)):
        if links[i].has_attr('target') is False:
            links[i]['target'] = '#note'+str(cnt)
            cnt += 1

    dictionary = {'а':'a','б':'b','в':'v','г':'g','д':'d','е':'e','ё':'yo',
          'ж':'zh','з':'z','и':'i','й':'i','к':'k','л':'l','м':'m','н':'n',
          'о':'o','п':'p','р':'r','с':'s','т':'t','у':'u','ф':'f','х':'kh',
          'ц':'c','ч':'ch','ш':'sh','щ':'sch','ъ':'','ы':'y','ь':'','э':'e',
          'ю':'u','я':'ya', 'А':'A','Б':'B','В':'V','Г':'G','Д':'D','Е':'E','Ё':'YO',
          'Ж':'ZH','З':'Z','И':'I','Й':'I','К':'K','Л':'L','М':'M','Н':'N',
          'О':'O','П':'P','Р':'R','С':'S','Т':'T','У':'U','Ф':'F','Х':'KH',
          'Ц':'C','Ч':'CH','Ш':'SH','Щ':'SCH','Ъ':'','Ы':'Y','Ь':'','Э':'E',
          'Ю':'U','Я':'YA', ' ' : '_'}

    title = re.sub(r'[/\:*?''<>!«»—,]', '', title_main_tei.string)
    for key in dictionary:
        title = title.replace(key, dictionary[key])
    title = title[:15]

    # Сохраняем
    file = open(new_directory + 'V3_'+ str(iter) + '_' + title + '.xml', 'w', encoding='utf8') # В зависимости от номера тома поменять цифру рядом с "V" (V1 - первый том)
    file.write(soup_tei.prettify())
    file.close()
    #print(soup_tei.prettify())

    iter += 1